## Textual visualization benchmark

In [1]:
using LinearAlgebra
using BenchmarkTools
using Plots

include("QuantumEvolution.jl")
using .QuantumEvolution

# Define the Hamiltonian and initial state
H = Complex{Float64}.([0 1; 1 0])  # Example Hamiltonian
ψ0 = Complex{Float64}.([1.0, 0.0])  # Initial state

# Define the time step and total time
Δt = 0.1
total_time = 1.0
num_steps = Int(total_time / Δt)

# Exact solution using matrix exponentiation
exact_solution(t) = exp(-1im * H * t) * ψ0
exact_solution_imag(τ) = exp(-H * τ) * ψ0

# Function to perform time evolution using a given method
function evolve(method::Function, ψ::Vector{ComplexF64}, H::Matrix{ComplexF64}, Δt::Float64, num_steps::Int)
    ψ_evolved = copy(ψ)
    for _ in 1:num_steps
        method(ψ_evolved, H, Δt)
    end
    return ψ_evolved
end

# Benchmark the methods
methods = [QuantumEvolution.euler_step!, QuantumEvolution.rk4_step!,
          (ψ, H, Δt) -> QuantumEvolution.trotter_step!(ψ, H, H, Δt, 10), QuantumEvolution.krylov_step!]
method_names = ["Euler", "RK4", "Trotter-Suzuki", "Krylov"]

imag_methods = [QuantumEvolution.euler_step_imag!, QuantumEvolution.rk4_step_imag!,
               (ψ, H, Δτ) -> QuantumEvolution.trotter_step_imag!(ψ, H, H, Δτ, 10), QuantumEvolution.krylov_step_imag!]
imag_method_names = ["Euler Imag", "RK4 Imag", "Trotter-Suzuki Imag", "Krylov Imag"]

println("Benchmarking Quantum Evolution Methods:")
results = []
imag_results = []

for (method, name) in zip(methods, method_names)
    println("Benchmarking $name method...")
    ψ_final = evolve(method, ψ0, H, Δt, num_steps)
    exact_ψ_final = exact_solution(total_time)
    error = norm(ψ_final - exact_ψ_final)
    time = @elapsed evolve(method, ψ0, H, Δt, num_steps)
    push!(results, (name, error, time))
    println("Error: $error, Time: $time seconds")
end

for (method, name) in zip(imag_methods, imag_method_names)
    println("Benchmarking $name method...")
    ψ_final_imag = evolve(method, ψ0, H, Δt, num_steps)
    exact_ψ_final_imag = exact_solution_imag(total_time)
    error_imag = norm(ψ_final_imag - exact_ψ_final_imag)
    time_imag = @elapsed evolve(method, ψ0, H, Δt, num_steps)
    push!(imag_results, (name, error_imag, time_imag))
    println("Error: $error_imag, Time: $time_imag seconds")
end


# Visualization
real_errors = [result[2] for result in results]
println(real_errors)
real_times = [result[3] for result in results]
println(real_times)
imag_errors = [result[2] for result in imag_results]
println(imag_errors)
imag_times = [result[3] for result in imag_results]
println(imag_times)

"""
plot(real_times, real_errors, xlabel="Time (s)", ylabel="Error", label="Real-Time Evolution", title="Real-Time Evolution Benchmark")
plot!(imag_times, imag_errors, xlabel="Time (s)", ylabel="Error", label="Imaginary-Time Evolution", title="Imaginary-Time Evolution Benchmark")
"""

Benchmarking Quantum Evolution Methods:
Benchmarking Euler method...
Error: 0.05112303163080999, Time: 1.2669e-5 seconds
Benchmarking RK4 method...
Error: 8.33250640984284e-7, Time: 2.0918e-5 seconds
Benchmarking Trotter-Suzuki method...
Error: 0.9588510772083934, Time: 0.001137162 seconds
Benchmarking Krylov method...
Error: 2.482534153247273e-16, Time: 4.1478e-5 seconds
Benchmarking Euler Imag method...
Error: 0.08910312203681531, Time: 9.474e-6 seconds
Benchmarking RK4 Imag method...
Error: 1.4925574750283086e-6, Time: 1.4198e-5 seconds
Benchmarking Trotter-Suzuki Imag method...
Error: 3.3068269588145545, Time: 0.000483718 seconds
Benchmarking Krylov Imag method...
Error: 6.280369834735101e-16, Time: 3.1006e-5 seconds
[0.05112303163080999, 8.33250640984284e-7, 0.9588510772083934, 2.482534153247273e-16]
[1.2669e-5, 2.0918e-5, 0.001137162, 4.1478e-5]
[0.08910312203681531, 1.4925574750283086e-6, 3.3068269588145545, 6.280369834735101e-16]
[9.474e-6, 1.4198e-5, 0.000483718, 3.1006e-5]


"plot(real_times, real_errors, xlabel=\"Time (s)\", ylabel=\"Error\", label=\"Real-Time Evolution\", title=\"Real-Time Evolution Benchmark\")\nplot!(imag_times, imag_errors, xlabel=\"Time (s)\", ylabel=\"Error\", label=\"Imaginary-Time Evolution\", title=\"Imaginary-Time Evolution Benchmark\")\n"

## Graphical visualization benchmark [v1]

In [6]:
using LinearAlgebra
using BenchmarkTools
#using QuantumEvolution
using Plots
using StatsPlots  # For additional plotting features
using Printf

include("QuantumEvolution.jl")
using .QuantumEvolution

"""
Structure to hold benchmark results
"""
struct BenchmarkResult
    method_name::String
    error::Float64
    execution_time::Float64
    energy_evolution::Vector{Float64}
    state_fidelity::Vector{Float64}
end

"""
Run benchmarks and collect detailed results
"""
function run_detailed_benchmark(H, ψ0, Δt, total_time)
    num_steps = Int(total_time / Δt)
    times = 0:Δt:total_time
    
    # Methods configuration
    methods = [
        (QuantumEvolution.euler_step!, "Euler"),
        (QuantumEvolution.rk4_step!, "RK4"),
        ((ψ, H, Δt) -> QuantumEvolution.trotter_step!(ψ, H, H, Δt, 10), "Trotter"),
        (QuantumEvolution.krylov_step!, "Krylov")
    ]
    
    imag_methods = [
        (QuantumEvolution.euler_step_imag!, "Euler Imag"),
        (QuantumEvolution.rk4_step_imag!, "RK4 Imag"),
        ((ψ, H, Δt) -> QuantumEvolution.trotter_step_imag!(ψ, H, H, Δt, 10), "Trotter Imag"),
        (QuantumEvolution.krylov_step_imag!, "Krylov Imag")
    ]
    
    real_results = Vector{BenchmarkResult}()
    imag_results = Vector{BenchmarkResult}()
    
    # Benchmark real-time methods
    for (method, name) in methods
        ψ = copy(ψ0)
        energy_evolution = Float64[]
        fidelity_evolution = Float64[]
        
        execution_time = @elapsed begin
            for t in times
                exact_ψ = exact_solution(t)
                push!(energy_evolution, real(dot(ψ, H * ψ)))
                push!(fidelity_evolution, abs2(dot(exact_ψ, ψ)))
                method(ψ, H, Δt)
            end
        end
        
        final_error = norm(ψ - exact_solution(total_time))
        push!(real_results, BenchmarkResult(name, final_error, execution_time,
                                          energy_evolution, fidelity_evolution))
    end
    
    # Benchmark imaginary-time methods
    for (method, name) in imag_methods
        ψ = copy(ψ0)
        energy_evolution = Float64[]
        fidelity_evolution = Float64[]
        
        execution_time = @elapsed begin
            for t in times
                exact_ψ = exact_solution_imag(t)
                push!(energy_evolution, real(dot(ψ, H * ψ)))
                push!(fidelity_evolution, abs2(dot(exact_ψ, ψ)))
                method(ψ, H, Δt)
                normalize!(ψ)
            end
        end
        
        final_error = norm(ψ - exact_solution_imag(total_time))
        push!(imag_results, BenchmarkResult(name, final_error, execution_time,
                                          energy_evolution, fidelity_evolution))
    end
    
    return real_results, imag_results, times
end

"""
Create comprehensive visualization of benchmark results
"""
function visualize_benchmark_results(real_results, imag_results, times)
    # 1. Performance Overview
    p1 = scatter(
        [r.execution_time for r in real_results],
        [r.error for r in real_results],
        xlabel="Execution Time (s)",
        ylabel="Final Error",
        label="Real-Time",
        title="Performance Overview",
        marker=:circle
    )
    scatter!(
        [r.execution_time for r in imag_results],
        [r.error for r in imag_results],
        label="Imaginary-Time",
        marker=:square
    )
    
    # 2. Energy Evolution
    p2 = plot(
        title="Energy Evolution",
        xlabel="Time",
        ylabel="Energy"
    )
    for result in real_results
        plot!(times, result.energy_evolution, label=result.method_name)
    end
    
    # 3. State Fidelity
    p3 = plot(
        title="State Fidelity",
        xlabel="Time",
        ylabel="Fidelity"
    )
    for result in real_results
        plot!(times, result.state_fidelity, label=result.method_name)
    end
    
    # 4. Error Bar Plot
    p4 = bar(
        getfield.(real_results, :method_name),
        [getfield.(real_results, :error) getfield.(imag_results, :error)],
        title="Final Error Comparison",
        xlabel="Method",
        ylabel="Error",
        label=["Real-Time" "Imaginary-Time"],
        bar_position=:dodge,
        rotation=45,
        legend=:topleft
    )

    # 5. Time Bar Plot
    p5 = bar(
        getfield.(real_results, :method_name),
        [getfield.(real_results, :execution_time) getfield.(imag_results, :execution_time)],
        title="Execution Time Comparison",
        xlabel="Method",
        ylabel="Time (s)",
        label=["Real-Time" "Imaginary-Time"],
        bar_position=:dodge,
        rotation=45,
        legend=:topleft
    )

    # Combine all plots
    final_plot = plot(p1, p3, p4, p5,
                     layout=(2,2),
                     size=(1200,800),
                     plot_title="Quantum Evolution Methods Benchmark")
    
    return final_plot
end

# Run benchmark
H = Complex{Float64}.([0 1; 1 0])
ψ0 = normalize(Complex{Float64}.([1.0, 0.0]))
Δt = 0.1
total_time = 1.0

real_results, imag_results, times = run_detailed_benchmark(H, ψ0, Δt, total_time)
benchmark_plot = visualize_benchmark_results(real_results, imag_results, times)
savefig(benchmark_plot, "quantum_evolution_benchmark.png")

benchmark_plot

# Print numerical results
println("\nNumerical Results Summary:")
println("\nReal-Time Methods:")
for result in real_results
    @printf("%s: Error = %.2e, Time = %.3f s\n",
            result.method_name, result.error, result.execution_time)
end

println("\nImaginary-Time Methods:")
for result in imag_results
    @printf("%s: Error = %.2e, Time = %.3f s\n",
            result.method_name, result.error, result.execution_time)
end


Numerical Results Summary:



Real-Time Methods:
Euler: Error = 1.14e-01, Time = 0.041 s
RK4: Error = 1.00e-01, Time = 0.059 s
Trotter: Error = 1.13e+00, Time = 0.051 s
Krylov: Error = 1.00e-01, Time = 0.039 s

Imaginary-Time Methods:
Euler Imag: Error = 9.40e-01, Time = 0.033 s
RK4 Imag: Error = 9.40e-01, Time = 0.044 s
Trotter Imag: Error = 9.55e-01, Time = 0.043 s
Krylov Imag: Error = 9.40e-01, Time = 0.045 s
